In [59]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import compose, pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn import compose, pipeline
from sklearn.model_selection import KFold
from typing import Optional
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import roc_auc_score, make_scorer, r2_score
from pandas import DataFrame, Series
import numpy as np
kf = KFold(n_splits=5, shuffle=True, random_state=42)
import xgboost as xgb
from xgboost import XGBRegressor
import optuna
from optuna import Trial
from optuna import create_study
from optuna import samplers
from sklearn import compose
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor

### Instantiate function for each regression model 

In [162]:
def instantiate_ridge(trial : Trial) -> Ridge:
    params = {
        "alpha": trial.suggest_float("alpha", 15, 25)
    }

    return Ridge(**params)

def instantiate_lasso(trial : Trial) -> Lasso:
    params = {
        "alpha": trial.suggest_float("alpha", .0001, .0004)
    }

    return Lasso(**params)

def instantiate_en(trial : Trial) -> ElasticNet:
    params = {
        "alpha": trial.suggest_float("alpha", .0001, .0004)
    }

    return ElasticNet(**params)

### Instantiate function for regression model selection

In [159]:
Classifier = (
    Ridge |
    Lasso |
    ElasticNet
    )

def instantiate_learner(trial : Trial) -> Classifier:
    algorithm = trial.suggest_categorical(
    'algorithm', ['ridge', 'lasso', 'en'])
    
    if algorithm =='ridge':
        model = instantiate_ridge(trial)
    elif algorithm=='lasso':
        model = instantiate_lasso(trial)
    elif algorithm=='en':
        model = instantiate_en(trial)
    return model

### Instantiate functions for encoding categorical columns 

In [144]:
from sklearn.preprocessing import OneHotEncoder
#from category_encoders import WOEEncoder

def instantiate_encoder(trial: Trial)-> OneHotEncoder:
    params = {
        'handle_unknown': 'ignore',
        'drop': trial.suggest_categorical('drop', [None, 'first'])
    }
    
    return OneHotEncoder(**params)
    

### Instantiate functions for encoding numerical columns 

In [145]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler

Scaler = (
  StandardScaler |
  MinMaxScaler |
  MaxAbsScaler |
  RobustScaler
)

def instantiate_scaler(trial : Trial) -> Scaler:
    method = trial.suggest_categorical(
    'scaling_method', ['standard', 'minmax', 'maxabs', 'robust']
    )
    if method=='standard':
        scaler = StandardScaler()
    elif method=='minmax':
        scaler = MinMaxScaler()
    elif method=='maxabs':
        scaler = MaxAbsScaler()
    elif method=='robust':
        scaler = RobustScaler()
        
    return scaler

### Instantiate function to scale and encode 

In [163]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

def instantiate_processor(trial : Trial, 
                          numerical_columns : list[str], 
                          categorical_columns : list[str]) -> ColumnTransformer:
    
    numerical_pipeline = instantiate_scaler(trial)
    categorical_pipeline = instantiate_encoder(trial)
   # numerical_pipeline = StandardScaler()
    #categorical_pipeline = OneHotEncoder(handle_unknown="ignore", drop='first')
    
#     numerical_pipeline = instantiate_numerical_pipeline(trial)
#     categorical_pipeline = instantiate_categorical_pipeline(trial)
    
    processor = ColumnTransformer([
        ('numerical_pipeline', numerical_pipeline, numerical_columns),
        ('categorical_pipeline', categorical_pipeline, categorical_columns)
    ])
    return processor

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
#     learner = instantiate_lasso(trial)
    learner = instantiate_learner(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

In [147]:
def objective(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [148]:
df = pd.read_csv('df_normal_quality.csv', index_col=0)
y = df.SalePrice
X = df.drop(['PID', 'SalePrice'], axis =1).copy()

In [172]:
from optuna import create_study
#study = create_study(sampler= samplers.RandomSampler(42), 

study = create_study(study_name='optimization', 
                     direction='maximize')

[I 2024-05-09 15:11:09,186] A new study created in memory with name: optimization


In [173]:
study.optimize(lambda trial: objective(trial, X, y), n_trials=500)

[I 2024-05-09 15:11:12,195] Trial 0 finished with value: 0.9499516590290001 and parameters: {'scaling_method': 'standard', 'drop': None, 'algorithm': 'en', 'alpha': 0.0002763555611934212}. Best is trial 0 with value: 0.9499516590290001.
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [3, 10] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categor

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros

[I 2024-05-09 15:11:16,885] Trial 7 finished with value: 0.9496567088711311 and parameters: {'scaling_method': 'standard', 'drop': 'first', 'algo

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros

[I 2024-05-09 15:11:33,030] Trial 35 finished with value: 0.9498795863160696 and parameters: {'scaling_method': 'robust', 'drop': 'first', 'algorithm': 'ridge', 'alpha': 15.087951305732666}. Best is trial 21 with value: 0.9508118984390123.
[I 2024-05-09 15:11:33,944] Trial 36 finished with value: 0.9508004856839587 and parameters: {'scaling_method': 'robust', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.0002843236065576017}. Best is trial 21 with value: 0.9508118984390123.
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:


[I 2024-05-09 15:11:42,290] Trial 50 finished with value: 0.9502243476716778 and parameters: {'scaling_method': 'standard', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.00036822778206654406}. Best is trial 44 with value: 0.9508124279335561.
[I 2024-05-09 15:11:43,093] Trial 51 finished with value: 0.9508098554531846 and parameters: {'scaling_method': 'robust', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.0003273967344593095}. Best is trial 44 with value: 0.9508124279335561.
[I 2024-05-09 15:11:43,906] Trial 52 finished with value: 0.9508005245886842 and parameters: {'scaling_method': 'robust', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.0003436421762786338}. Best is trial 44 with value: 0.9508124279335561.
[I 2024-05-09 15:11:44,787] Trial 53 finished with value: 0.9508117711095098 and parameters: {'scaling_method': 'robust', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.00030165544892913964}. Best is trial 44 with value: 0.9508124279335561.
[I 2024-05-09 15:11:45,658] Trial 54

[I 2024-05-09 15:12:06,317] Trial 80 finished with value: 0.9499545416246533 and parameters: {'scaling_method': 'robust', 'drop': None, 'algorithm': 'ridge', 'alpha': 15.025583457740456}. Best is trial 74 with value: 0.9508124950361158.
[I 2024-05-09 15:12:07,143] Trial 81 finished with value: 0.9508122884639405 and parameters: {'scaling_method': 'robust', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.0003066623528944073}. Best is trial 74 with value: 0.9508124950361158.
[I 2024-05-09 15:12:07,973] Trial 82 finished with value: 0.9508117299963101 and parameters: {'scaling_method': 'robust', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.0003101556944283647}. Best is trial 74 with value: 0.9508124950361158.
[I 2024-05-09 15:12:08,789] Trial 83 finished with value: 0.9508049299456391 and parameters: {'scaling_method': 'robust', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.00033717644211051114}. Best is trial 74 with value: 0.9508124950361158.
[I 2024-05-09 15:12:09,615] Trial 84 finis

...

[I 2024-05-09 15:15:40,547] Trial 367 finished with value: 0.9508093956333943 and parameters: {'scaling_method': 'robust', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.000294246100155041}. Best is trial 131 with value: 0.9508125921535637.
[I 2024-05-09 15:15:41,395] Trial 368 finished with value: 0.9508106740701244 and parameters: {'scaling_method': 'robust', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.0003129643289125872}. Best is trial 131 with value: 0.9508125921535637.
[I 2024-05-09 15:15:42,337] Trial 369 finished with value: 0.9508033832796444 and parameters: {'scaling_method': 'robust', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.0002871553759229408}. Best is trial 131 with value: 0.9508125921535637.
[I 2024-05-09 15:15:43,148] Trial 370 finished with value: 0.950811929569815 and parameters: {'scaling_method': 'robust', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.00030223205060260857}. Best is trial 131 with value: 0.9508125921535637.
[I 2024-05-09 15:15:43,896] Trial

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [3, 10] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [0] duri

[I 2024-05-09 15:16:15,071] Trial 422 finished with value: 0.9508111916596075 and parameters: {'scaling_method': 'robust', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.0002985237141023597}. Best is trial 131 with value: 0.9508125921535637.
[I 2024-05-09 15:16:15,816] Trial 423 finished with value: 0.9502786457751032 and parameters: {'scaling_method': 'standard', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.0003164626358160414}. Best is trial 131 with value: 0.9508125921535637.
[I 2024-05-09 15:16:16,732] Trial 424 finished with value: 0.9508077124023824 and parameters: {'scaling_method': 'robust', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.00029165411390804334}. Best is trial 131 with value: 0.9508125921535637.
[I 2024-05-09 15:16:17,676] Trial 425 finished with value: 0.950812313736486 and parameters: {'scaling_method': 'robust', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.00030639313192955557}. Best is trial 131 with value: 0.9508125921535637.
[I 2024-05-09 15:16:18,575] T

[I 2024-05-09 15:16:37,991] Trial 452 finished with value: 0.9499540811163747 and parameters: {'scaling_method': 'robust', 'drop': None, 'algorithm': 'ridge', 'alpha': 15.00986626023484}. Best is trial 131 with value: 0.9508125921535637.
[I 2024-05-09 15:16:38,813] Trial 453 finished with value: 0.9508122810094026 and parameters: {'scaling_method': 'robust', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.0003067390679521191}. Best is trial 131 with value: 0.9508125921535637.
[I 2024-05-09 15:16:39,640] Trial 454 finished with value: 0.9508113409566393 and parameters: {'scaling_method': 'robust', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.0003172368307717228}. Best is trial 131 with value: 0.9508125921535637.
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning:

Found unknown categories in columns [3, 10] during transform. These unknown categories will be encoded as all zeros

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing

[I 2024-05-09 15:16:58,690] Trial 477 finished with value: 0.9508124479961717 and parameters: {'scaling_method': 'robust', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.00030499902028138055}. Best is trial 131 with value: 0.9508125921535637.
[I 2024-05-09 15:16:59,522] Trial 478 finished with value: 0.9508107485260073 and parameters: {'scaling_method': 'robust', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.00029700698962539085}. Best is trial 131 with value: 0.9508125921535637.
[I 2024-05-09 15:17:00,381] Trial 479 finished with value: 0.9508119594223926 and parameters: {'scaling_method': 'robust', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.000302346113489379}. Best is trial 131 with value: 0.9508125921535637.
[I 2024-05-09 15:17:01,792] Trial 480 finished with value: 0.9473481738795944 and parameters: {'scaling_method': 'maxabs', 'drop': None, 'algorithm': 'en', 'alpha': 0.00010035118104258578}. Best is trial 131 with value: 0.9508125921535637.
[I 2024-05-09 15:17:02,157] Trial

In [174]:
study.best_trial
#[0.9508125905283533]

FrozenTrial(number=497, state=1, values=[0.9508125946535184], datetime_start=datetime.datetime(2024, 5, 9, 15, 17, 12, 820009), datetime_complete=datetime.datetime(2024, 5, 9, 15, 17, 13, 693235), params={'scaling_method': 'robust', 'drop': None, 'algorithm': 'lasso', 'alpha': 0.0003061900989342117}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'scaling_method': CategoricalDistribution(choices=('standard', 'minmax', 'maxabs', 'robust')), 'drop': CategoricalDistribution(choices=(None, 'first')), 'algorithm': CategoricalDistribution(choices=('ridge', 'lasso', 'en')), 'alpha': FloatDistribution(high=0.0004, log=False, low=0.0001, step=None)}, trial_id=497, value=None)

In [175]:
trial_number = []
score = []
algorithm = []
parameters = []
optuna_dict = {}
trials = range(500)

for trial in trials:
    trial_number.append(study.get_trials()[trial].number)
    score.append(study.get_trials()[trial].value)
    algorithm.append(study.get_trials()[trial].params['algorithm'])
    parameters.append(list(study.get_trials()[trial].params.items()))

optuna_dict['Trial'] = trial_number
optuna_dict['Score'] = score
optuna_dict['Algorithm'] = algorithm
optuna_dict['Parameters'] = parameters


df_optuna = pd.DataFrame.from_dict(optuna_dict)

In [177]:
idx = df_optuna.groupby('Algorithm')['Score'].transform(max) == df_optuna['Score']
df_optuna[idx]

,Trial,Score,Algorithm,Parameters
8,8,0.950430,en,"[(scaling_method, robust), (drop, None), (algo..."
80,80,0.949955,ridge,"[(scaling_method, robust), (drop, None), (algo..."
497,497,0.950813,lasso,"[(scaling_method, robust), (drop, None), (algo..."


In [179]:
print('lasso', df_optuna.Parameters[497])
print('ridge', df_optuna.Parameters[80])
print('en', df_optuna.Parameters[8])

lasso [('scaling_method', 'robust'), ('drop', None), ('algorithm', 'lasso'), ('alpha', 0.0003061900989342117)]
ridge [('scaling_method', 'robust'), ('drop', None), ('algorithm', 'ridge'), ('alpha', 15.025583457740456)]
en [('scaling_method', 'robust'), ('drop', None), ('algorithm', 'en'), ('alpha', 0.00035734815701707963)]


In [178]:
import plotly.express as px

#fig = px.scatter(df_optuna.loc[df_optuna.Score > .5], 
fig = px.scatter(df_optuna, 
                 x="Trial", 
                 y="Score", 
                 color="Algorithm",
                 hover_data=['Parameters'])

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)

fig.show()

In [30]:
import plotly.graph_objects as go

fig = optuna.visualization.plot_rank(study)

fig.update_layout(
    autosize=False,
    width=1000,
    height=1000,
    )

fig.show()

C:\Users\Natal\AppData\Local\Temp\ipykernel_24588\2598969432.py:3: ExperimentalWarning:

plot_rank is experimental (supported from v3.2.0). The interface can change in the future.

